In [4]:
#annual income jumped indicating the enterprise was flying away from valley.

import aushare.stock.fundamental as td
from datetime import datetime
import pandas as pd

CRITERIA_THRELD_VALUE = 0.5
outstand = []
asxcodelist = td.getAllASXListCode().unique()

#asxcodelist =['IRE','APT','Z1P','APX','MOQ','1PG','ONT','14D']
#asxcodelist = ['T3D','TGP']
for asxcode in asxcodelist:
    df4 = td.getRevenueDiff(asxcode)
    if isinstance(df4,type(None)):
        continue
    #print (df4)
    df41 = df4[df4['difference']>=CRITERIA_THRELD_VALUE]
    print(df41)
    if not df41.empty:
            tmpdict = {'Asx Code':asxcode, 'Finacial Year':df41['date'].iloc[-1],'Revenue Increase %':round(df41['difference'].iloc[-1].astype(float),2)*100}
            print (df41.shape[0])
            
            startyear = datetime.strptime(df41['date'].iloc[-1], '%m/%d/%Y').year
            print('staryear=',startyear)
            
            yearafter = startyear+1
            
            pricediff = 0 if datetime.now().year==startyear else td.getMeanPriceDiffPercentage(asxcode,startyear, yearafter)
            
            print(pricediff)
            if pricediff is None:
                print("no price difference can be found for code %s"%asxcode)
                continue
            tmpdict.update({'pricediff':pricediff})
            df_basicinfo= td.getCompanyBasicInfo(asxcode)
    
            tmpdict.update({'Company Name':df_basicinfo['Company name'].values[0], 'industry':df_basicinfo['GICS industry group'].values[0]})
            
            print(tmpdict)
            outstand.append(tmpdict)

outstandshares = pd.DataFrame(outstand)
outstandshares.to_csv('shares_likely_jump.csv')
            
    

https://finance.yahoo.com/quote/IRE.AX/financials?p=IRE.AX
         date  difference
0  12/31/2017    0.103185
1  12/31/2016    0.078218
2  12/31/2015    0.098431
3  12/31/2014         NaN
Empty DataFrame
Columns: [date, difference]
Index: []
https://finance.yahoo.com/quote/APT.AX/financials?p=APT.AX
        date  difference
0  6/30/2018    3.972453
1  6/30/2017   15.559660
2  6/30/2016   57.004822
3  6/30/2015         NaN
        date  difference
0  6/30/2018    3.972453
1  6/30/2017   15.559660
2  6/30/2016   57.004822
3
staryear= 2016
https://au.finance.yahoo.com/quote/APT.AX/history?period1=1451566800&period2=1483102800&interval=1wk&filter=history&frequency=1wk
nan
https://au.finance.yahoo.com/quote/APT.AX/history?period1=1483189200&period2=1514638800&interval=1wk&filter=history&frequency=1wk
4.322592592592593
nan
    GICS industry group                  Company name
59  Software & Services  AFTERPAY TOUCH GROUP LIMITED
{'Asx Code': 'APT', 'Finacial Year': '6/30/2016', 'Revenue Inc

In [1]:
#filtered by debt/asset ratio criteria

import aushare.stock.fundamental as td
from datetime import datetime
import pandas as pd

DEBT_ASSET_RATIO_CRITERIA = 0.6

def get_liabilities_assets_ratio(code,date):
    df=td.getBalanceSheet(code)
    dateobject = datetime.strptime(date,'%m/%d/%Y')
    date = dateobject.strftime('%d/%m/%Y')

    assets = round(float(df['Total assets'].loc[date]),2)
    debts= round(float(df['Total liabilities'].loc[date]),2)
    return 0 if assets ==0 else round(debts/assets,2)

dataFile = './shares_likely_jump.csv'
df1 =pd.read_csv(dataFile,header=0,index_col=0)

df = df1.set_index('Asx Code',drop=False)
print(df)
for code in df['Asx Code']:
    print ('ASX Code =',code)
    try:
        #print('xxxxxyyy',df['Finacial Year'].loc[df['Asx Code']==code].iloc[0])
        #print('yyyyyyyy',df['Finacial Year'].loc[df['Asx Code']==code].iloc[0])
        date = df['Finacial Year'].loc[df['Asx Code']==code].iloc[0].strip() 
        ratio = get_liabilities_assets_ratio(code,date)
        print('code={0} date={1} ratio={2}'.format(code,date,ratio))
        if ratio > DEBT_ASSET_RATIO_CRITERIA:
            print('code {0} debt asset ratio = {1} and dropped'.format(code,ratio))
            df.drop([code],inplace=True)    
    except:
        print('no information.')
        
print(df)

df.to_csv('shares_likely_jump_debt_asset_filter.csv')
    



         Asx Code                                      Company Name  \
Asx Code                                                              
MOQ           MOQ                                       MOQ LIMITED   
1ST           1ST                                 1ST GROUP LIMITED   
T3D           T3D                                      333D LIMITED   
TGP           TGP                                 360 CAPITAL GROUP   
TDO           TDO                                    3D OIL LIMITED   
4DS           4DS                                4DS MEMORY LIMITED   
5GN           5GN                               5G NETWORKS LIMITED   
8IH           8IH                                   8I HOLDINGS LTD   
9SP           9SP                    9 SPOKES INTERNATIONAL LIMITED   
ACB           ACB                           A-CAP RESOURCES LIMITED   
AYI           AYI                    A1 INVESTMENTS & RESOURCES LTD   
ABL           ABL                       ABILENE OIL AND GAS LIMITED   
ABT   

code=MOQ date=6/30/2016 ratio=0.33
ASX Code = 1ST
https://au.finance.yahoo.com/quote/1ST.AX/balance-sheet?p=1ST.AX
code=1ST date=6/30/2016 ratio=0.11
ASX Code = T3D
https://au.finance.yahoo.com/quote/T3D.AX/balance-sheet?p=T3D.AX
code=T3D date=6/30/2016 ratio=2.94
code T3D debt asset ratio = 2.94 and dropped
ASX Code = TGP
https://au.finance.yahoo.com/quote/TGP.AX/balance-sheet?p=TGP.AX
code=TGP date=6/30/2016 ratio=0.49
ASX Code = TDO
https://au.finance.yahoo.com/quote/TDO.AX/balance-sheet?p=TDO.AX
code=TDO date=6/30/2015 ratio=0.06
ASX Code = 4DS
https://au.finance.yahoo.com/quote/4DS.AX/balance-sheet?p=4DS.AX
code=4DS date=6/30/2016 ratio=0.07
ASX Code = 5GN
https://au.finance.yahoo.com/quote/5GN.AX/balance-sheet?p=5GN.AX
code=5GN date=6/30/2018 ratio=0.37
ASX Code = 8IH
https://au.finance.yahoo.com/quote/8IH.AX/balance-sheet?p=8IH.AX
code=8IH date=3/31/2016 ratio=0.12
ASX Code = 9SP
https://au.finance.yahoo.com/quote/9SP.AX/balance-sheet?p=9SP.AX
code=9SP date=3/31/2016 ratio=0.79


code=ACP date=6/30/2015 ratio=0.48
ASX Code = AD8
https://au.finance.yahoo.com/quote/AD8.AX/balance-sheet?p=AD8.AX
code=AD8 date=6/30/2016 ratio=0.25
ASX Code = AU8
https://au.finance.yahoo.com/quote/AU8.AX/balance-sheet?p=AU8.AX
code=AU8 date=6/30/2018 ratio=0.16
ASX Code = AEE
https://au.finance.yahoo.com/quote/AEE.AX/balance-sheet?p=AEE.AX
code=AEE date=6/30/2015 ratio=0.04
ASX Code = AMI
https://au.finance.yahoo.com/quote/AMI.AX/balance-sheet?p=AMI.AX
code=AMI date=6/30/2016 ratio=1.21
code AMI debt asset ratio = 1.21 and dropped
ASX Code = AOU
https://au.finance.yahoo.com/quote/AOU.AX/balance-sheet?p=AOU.AX
code=AOU date=6/30/2015 ratio=0.5
ASX Code = ABW
https://au.finance.yahoo.com/quote/ABW.AX/balance-sheet?p=ABW.AX
code=ABW date=6/30/2016 ratio=0.01
ASX Code = ARM
https://au.finance.yahoo.com/quote/ARM.AX/balance-sheet?p=ARM.AX
code=ARM date=6/30/2016 ratio=0.02
ASX Code = AQD
https://au.finance.yahoo.com/quote/AQD.AX/balance-sheet?p=AQD.AX
code=AQD date=6/30/2017 ratio=0.36
A

code=BMP date=6/30/2017 ratio=0.29
ASX Code = BWX
https://au.finance.yahoo.com/quote/BWX.AX/balance-sheet?p=BWX.AX
code=BWX date=6/30/2016 ratio=0.15
ASX Code = BPG
https://au.finance.yahoo.com/quote/BPG.AX/balance-sheet?p=BPG.AX
no information.
ASX Code = CXL
https://au.finance.yahoo.com/quote/CXL.AX/balance-sheet?p=CXL.AX
no information.
ASX Code = CAN
https://au.finance.yahoo.com/quote/CAN.AX/balance-sheet?p=CAN.AX
code=CAN date=6/30/2018 ratio=0.01
ASX Code = CAE
https://au.finance.yahoo.com/quote/CAE.AX/balance-sheet?p=CAE.AX
code=CAE date=6/30/2017 ratio=0.59
ASX Code = CAQ
https://au.finance.yahoo.com/quote/CAQ.AX/balance-sheet?p=CAQ.AX
code=CAQ date=12/31/2015 ratio=0.09
ASX Code = CVV
https://au.finance.yahoo.com/quote/CVV.AX/balance-sheet?p=CVV.AX
code=CVV date=6/30/2016 ratio=0.07
ASX Code = CRB
https://au.finance.yahoo.com/quote/CRB.AX/balance-sheet?p=CRB.AX
code=CRB date=12/31/2017 ratio=0.06
ASX Code = CG1
https://au.finance.yahoo.com/quote/CG1.AX/balance-sheet?p=CG1.AX
c

code=DHR date=6/30/2015 ratio=0.32
ASX Code = DTM
https://au.finance.yahoo.com/quote/DTM.AX/balance-sheet?p=DTM.AX
code=DTM date=6/30/2016 ratio=0.04
ASX Code = DEG
https://au.finance.yahoo.com/quote/DEG.AX/balance-sheet?p=DEG.AX
code=DEG date=6/30/2017 ratio=0.47
ASX Code = DSX
https://au.finance.yahoo.com/quote/DSX.AX/balance-sheet?p=DSX.AX
code=DSX date=6/30/2016 ratio=0.22
ASX Code = D13
https://au.finance.yahoo.com/quote/D13.AX/balance-sheet?p=D13.AX
code=D13 date=6/30/2017 ratio=0.12
ASX Code = DGH
https://au.finance.yahoo.com/quote/DGH.AX/balance-sheet?p=DGH.AX
code=DGH date=6/30/2016 ratio=0.43
ASX Code = DXS
https://au.finance.yahoo.com/quote/DXS.AX/balance-sheet?p=DXS.AX
code=DXS date=6/30/2015 ratio=0.32
ASX Code = DGR
https://au.finance.yahoo.com/quote/DGR.AX/balance-sheet?p=DGR.AX
code=DGR date=6/30/2016 ratio=0.07
ASX Code = DMC
https://au.finance.yahoo.com/quote/DMC.AX/balance-sheet?p=DMC.AX
code=DMC date=3/31/2018 ratio=0.39
ASX Code = DXB
https://au.finance.yahoo.com/q

...

code=MYX date=6/30/2016 ratio=0.75
code MYX debt asset ratio = 0.75 and dropped
ASX Code = MEA
https://au.finance.yahoo.com/quote/MEA.AX/balance-sheet?p=MEA.AX
code=MEA date=6/30/2016 ratio=0.28
ASX Code = MMR
https://au.finance.yahoo.com/quote/MMR.AX/balance-sheet?p=MMR.AX
code=MMR date=6/30/2017 ratio=0.06
ASX Code = MDR
https://au.finance.yahoo.com/quote/MDR.AX/balance-sheet?p=MDR.AX
code=MDR date=6/30/2017 ratio=0.16
ASX Code = MEB
https://au.finance.yahoo.com/quote/MEB.AX/balance-sheet?p=MEB.AX
code=MEB date=6/30/2016 ratio=0.53
ASX Code = MVP
https://au.finance.yahoo.com/quote/MVP.AX/balance-sheet?p=MVP.AX
code=MVP date=6/30/2016 ratio=0.54
ASX Code = MGZ
https://au.finance.yahoo.com/quote/MGZ.AX/balance-sheet?p=MGZ.AX
code=MGZ date=6/30/2016 ratio=2.72
code MGZ debt asset ratio = 2.72 and dropped
ASX Code = MDC
https://au.finance.yahoo.com/quote/MDC.AX/balance-sheet?p=MDC.AX
code=MDC date=6/30/2016 ratio=0.42
ASX Code = MP1
https://au.finance.yahoo.com/quote/MP1.AX/balance-sheet

code=NBL date=7/2/2017 ratio=0.65
code NBL debt asset ratio = 0.65 and dropped
ASX Code = NTU
https://au.finance.yahoo.com/quote/NTU.AX/balance-sheet?p=NTU.AX
code=NTU date=6/30/2015 ratio=1.2
code NTU debt asset ratio = 1.2 and dropped
ASX Code = NOR
https://au.finance.yahoo.com/quote/NOR.AX/balance-sheet?p=NOR.AX
code=NOR date=6/30/2016 ratio=0.22
ASX Code = NOV
https://au.finance.yahoo.com/quote/NOV.AX/balance-sheet?p=NOV.AX
code=NOV date=6/30/2016 ratio=0.34
ASX Code = NHL
https://au.finance.yahoo.com/quote/NHL.AX/balance-sheet?p=NHL.AX
code=NHL date=6/30/2017 ratio=0.11
ASX Code = NWH
https://au.finance.yahoo.com/quote/NWH.AX/balance-sheet?p=NWH.AX
code=NWH date=6/30/2018 ratio=0.48
ASX Code = NSX
https://au.finance.yahoo.com/quote/NSX.AX/balance-sheet?p=NSX.AX
code=NSX date=6/30/2017 ratio=0.21
ASX Code = NCR
https://au.finance.yahoo.com/quote/NCR.AX/balance-sheet?p=NCR.AX
code=NCR date=6/30/2015 ratio=0.14
ASX Code = OGA
https://au.finance.yahoo.com/quote/OGA.AX/balance-sheet?p=

code=QMS date=6/30/2016 ratio=0.29
ASX Code = QST
https://au.finance.yahoo.com/quote/QST.AX/balance-sheet?p=QST.AX
no information.
ASX Code = QVE
https://au.finance.yahoo.com/quote/QVE.AX/balance-sheet?p=QVE.AX
code=QVE date=6/30/2016 ratio=0.06
ASX Code = RMS
https://au.finance.yahoo.com/quote/RMS.AX/balance-sheet?p=RMS.AX
code=RMS date=6/30/2016 ratio=0.34
ASX Code = RBR
https://au.finance.yahoo.com/quote/RBR.AX/balance-sheet?p=RBR.AX
code=RBR date=6/30/2016 ratio=0.44
ASX Code = RCR
https://au.finance.yahoo.com/quote/RCR.AX/balance-sheet?p=RCR.AX
code=RCR date=6/30/2017 ratio=0.6
ASX Code = RCL
https://au.finance.yahoo.com/quote/RCL.AX/balance-sheet?p=RCL.AX
no information.
ASX Code = RLE
https://au.finance.yahoo.com/quote/RLE.AX/balance-sheet?p=RLE.AX
code=RLE date=6/30/2015 ratio=0.05
ASX Code = RCE
https://au.finance.yahoo.com/quote/RCE.AX/balance-sheet?p=RCE.AX
code=RCE date=6/30/2016 ratio=0.06
ASX Code = RFT
https://au.finance.yahoo.com/quote/RFT.AX/balance-sheet?p=RFT.AX
code

code=SM8 date=6/30/2016 ratio=0.12
ASX Code = SPZ
https://au.finance.yahoo.com/quote/SPZ.AX/balance-sheet?p=SPZ.AX
code=SPZ date=6/30/2016 ratio=0.36
ASX Code = SIQ
https://au.finance.yahoo.com/quote/SIQ.AX/balance-sheet?p=SIQ.AX
code=SIQ date=12/31/2016 ratio=0.56
ASX Code = SXX
https://au.finance.yahoo.com/quote/SXX.AX/balance-sheet?p=SXX.AX
code=SXX date=6/30/2015 ratio=0.1
ASX Code = SVM
https://au.finance.yahoo.com/quote/SVM.AX/balance-sheet?p=SVM.AX
code=SVM date=6/30/2017 ratio=0.04
ASX Code = SP3
https://au.finance.yahoo.com/quote/SP3.AX/balance-sheet?p=SP3.AX
code=SP3 date=6/30/2016 ratio=0.81
code SP3 debt asset ratio = 0.81 and dropped
ASX Code = ST1
https://au.finance.yahoo.com/quote/ST1.AX/balance-sheet?p=ST1.AX
code=ST1 date=6/30/2016 ratio=0.56
ASX Code = SIX
https://au.finance.yahoo.com/quote/SIX.AX/balance-sheet?p=SIX.AX
code=SIX date=6/30/2016 ratio=1.01
code SIX debt asset ratio = 1.01 and dropped
ASX Code = SPL
https://au.finance.yahoo.com/quote/SPL.AX/balance-sheet

code=UBI date=12/31/2015 ratio=0.71
code UBI debt asset ratio = 0.71 and dropped
ASX Code = UNV
https://au.finance.yahoo.com/quote/UNV.AX/balance-sheet?p=UNV.AX
code=UNV date=6/30/2016 ratio=0.42
ASX Code = URF
https://au.finance.yahoo.com/quote/URF.AX/balance-sheet?p=URF.AX
code=URF date=12/31/2015 ratio=0.52
ASX Code = UCM
https://au.finance.yahoo.com/quote/UCM.AX/balance-sheet?p=UCM.AX
code=UCM date=6/30/2016 ratio=0.14
ASX Code = VMX
https://au.finance.yahoo.com/quote/VMX.AX/balance-sheet?p=VMX.AX
code=VMX date=6/30/2017 ratio=0.59
ASX Code = VAN
https://au.finance.yahoo.com/quote/VAN.AX/balance-sheet?p=VAN.AX
code=VAN date=6/30/2015 ratio=0.14
ASX Code = VAR
https://au.finance.yahoo.com/quote/VAR.AX/balance-sheet?p=VAR.AX
code=VAR date=6/30/2015 ratio=0.1
ASX Code = VMG
https://au.finance.yahoo.com/quote/VMG.AX/balance-sheet?p=VMG.AX
code=VMG date=6/30/2017 ratio=1.01
code VMG debt asset ratio = 1.01 and dropped
ASX Code = VP7
https://au.finance.yahoo.com/quote/VP7.AX/balance-shee

In [1]:
#filtered by cash burn

import aushare.stock.fundamental as td
from datetime import datetime
import pandas as pd

#This means the remained cash can sustain to be burned for another 2 years with current cash burn speed.
CASHBURN_CRITERIA = 1 
dataFile = './shares_likely_jump_debt_asset_filter.csv'
df2 =pd.read_csv(dataFile,header=0,index_col=0)
#df2.drop(['Asx Code.1'],axis=1,inplace=True)
df2.rename(columns={"Asx Code.1": "Asx Code"},inplace=True)


print (df2.head)

for code in df2['Asx Code']:
    code_tmp = code
    try:
        date = df2['Finacial Year'].loc[df2['Asx Code']==code].iloc[0].strip()
        dateobject = datetime.strptime(date,'%m/%d/%Y')
        date = dateobject.strftime('%d/%m/%Y')
        df=td.getCashflow(code)
        cashburn = round(float(df['Change in cash and cash equivalents'].loc[date]),2)
        print('cashburn=',cashburn)
        df=td.getBalanceSheet(code)
        curcash = round(float(df['Cash and cash equivalents'].loc[date]),2)
        print('curcash=',curcash)
        if (cashburn<0) and (curcash/abs(cashburn))<CASHBURN_CRITERIA:
            print('code {0} cashburn too much and dropped'.format(code))
            df2.drop([code],inplace=True)
    except:
        print('no information in cash flow report.')
        df2.drop([code_tmp],inplace=True)
    
print(df2)
df2.to_csv('shares_likely_jump_cashburn_filter.csv')


<bound method NDFrame.head of          Asx Code                                      Company Name  \
Asx Code                                                              
MOQ           MOQ                                       MOQ LIMITED   
1ST           1ST                                 1ST GROUP LIMITED   
TGP           TGP                                 360 CAPITAL GROUP   
TDO           TDO                                    3D OIL LIMITED   
4DS           4DS                                4DS MEMORY LIMITED   
5GN           5GN                               5G NETWORKS LIMITED   
8IH           8IH                                   8I HOLDINGS LTD   
ACB           ACB                           A-CAP RESOURCES LIMITED   
AYI           AYI                    A1 INVESTMENTS & RESOURCES LTD   
ABL           ABL                       ABILENE OIL AND GAS LIMITED   
ABT           ABT                          ABUNDANT PRODUCE LIMITED   
AX1           AX1                              

cashburn= 356.03
https://au.finance.yahoo.com/quote/MOQ.AX/balance-sheet?p=MOQ.AX
curcash= 3078.33
https://au.finance.yahoo.com/quote/1ST.AX/cash-flow?p=1ST.AX
cashburn= 3141.28
https://au.finance.yahoo.com/quote/1ST.AX/balance-sheet?p=1ST.AX
curcash= 6563.83
https://au.finance.yahoo.com/quote/TGP.AX/cash-flow?p=TGP.AX
cashburn= -1235.0
https://au.finance.yahoo.com/quote/TGP.AX/balance-sheet?p=TGP.AX
curcash= 13732.0
https://au.finance.yahoo.com/quote/TDO.AX/cash-flow?p=TDO.AX
cashburn= 9666.53
https://au.finance.yahoo.com/quote/TDO.AX/balance-sheet?p=TDO.AX
curcash= 10494.4
https://au.finance.yahoo.com/quote/4DS.AX/cash-flow?p=4DS.AX
cashburn= 1234.38
https://au.finance.yahoo.com/quote/4DS.AX/balance-sheet?p=4DS.AX
curcash= 1243.49
https://au.finance.yahoo.com/quote/5GN.AX/cash-flow?p=5GN.AX
cashburn= 3117.9
https://au.finance.yahoo.com/quote/5GN.AX/balance-sheet?p=5GN.AX
curcash= 3356.7
https://au.finance.yahoo.com/quote/8IH.AX/cash-flow?p=8IH.AX
cashburn= -2919.34
https://au.finance

curcash= 142.84
code AMD cashburn too much and dropped
https://au.finance.yahoo.com/quote/ARV.AX/cash-flow?p=ARV.AX
cashburn= -410.57
https://au.finance.yahoo.com/quote/ARV.AX/balance-sheet?p=ARV.AX
curcash= 340.92
code ARV cashburn too much and dropped
https://au.finance.yahoo.com/quote/AFA.AX/cash-flow?p=AFA.AX
cashburn= -1732.0
https://au.finance.yahoo.com/quote/AFA.AX/balance-sheet?p=AFA.AX
curcash= 2497.0
https://au.finance.yahoo.com/quote/AJJ.AX/cash-flow?p=AJJ.AX
cashburn= 2709.38
https://au.finance.yahoo.com/quote/AJJ.AX/balance-sheet?p=AJJ.AX
curcash= 6249.37
https://au.finance.yahoo.com/quote/APZ.AX/cash-flow?p=APZ.AX
cashburn= -25390.0
https://au.finance.yahoo.com/quote/APZ.AX/balance-sheet?p=APZ.AX
curcash= 18237.0
code APZ cashburn too much and dropped
https://au.finance.yahoo.com/quote/ATR.AX/cash-flow?p=ATR.AX
cashburn= -3787.36
https://au.finance.yahoo.com/quote/ATR.AX/balance-sheet?p=ATR.AX
curcash= 1317.23
code ATR cashburn too much and dropped
https://au.finance.yaho

cashburn= 722.0
https://au.finance.yahoo.com/quote/BKL.AX/balance-sheet?p=BKL.AX
curcash= 37653.0
https://au.finance.yahoo.com/quote/BWF.AX/cash-flow?p=BWF.AX
cashburn= -1277.0
https://au.finance.yahoo.com/quote/BWF.AX/balance-sheet?p=BWF.AX
curcash= 688.0
code BWF cashburn too much and dropped
https://au.finance.yahoo.com/quote/BWR.AX/cash-flow?p=BWR.AX
cashburn= 1838.0
https://au.finance.yahoo.com/quote/BWR.AX/balance-sheet?p=BWR.AX
curcash= 2154.0
https://au.finance.yahoo.com/quote/BCT.AX/cash-flow?p=BCT.AX
cashburn= -254.87
https://au.finance.yahoo.com/quote/BCT.AX/balance-sheet?p=BCT.AX
curcash= 487.93
https://au.finance.yahoo.com/quote/BLG.AX/cash-flow?p=BLG.AX
cashburn= 5101.23
https://au.finance.yahoo.com/quote/BLG.AX/balance-sheet?p=BLG.AX
curcash= 8510.93
https://au.finance.yahoo.com/quote/BMG.AX/cash-flow?p=BMG.AX
cashburn= 1361.34
https://au.finance.yahoo.com/quote/BMG.AX/balance-sheet?p=BMG.AX
curcash= 1390.4
https://au.finance.yahoo.com/quote/BDA.AX/cash-flow?p=BDA.AX
cas

cashburn= 9229.0
https://au.finance.yahoo.com/quote/CL1.AX/balance-sheet?p=CL1.AX
curcash= 15179.0
https://au.finance.yahoo.com/quote/CSS.AX/cash-flow?p=CSS.AX
cashburn= -915.0
https://au.finance.yahoo.com/quote/CSS.AX/balance-sheet?p=CSS.AX
curcash= 598.0
code CSS cashburn too much and dropped
https://au.finance.yahoo.com/quote/CLQ.AX/cash-flow?p=CLQ.AX
cashburn= 3913.0
https://au.finance.yahoo.com/quote/CLQ.AX/balance-sheet?p=CLQ.AX
curcash= 7226.0
https://au.finance.yahoo.com/quote/CVW.AX/cash-flow?p=CVW.AX
cashburn= 4524.0
https://au.finance.yahoo.com/quote/CVW.AX/balance-sheet?p=CVW.AX
curcash= 222197.0
https://au.finance.yahoo.com/quote/CUV.AX/cash-flow?p=CUV.AX
cashburn= 3272.41
https://au.finance.yahoo.com/quote/CUV.AX/balance-sheet?p=CUV.AX
curcash= 13844.7
https://au.finance.yahoo.com/quote/CLV.AX/cash-flow?p=CLV.AX
cashburn= -1321.0
https://au.finance.yahoo.com/quote/CLV.AX/balance-sheet?p=CLV.AX
curcash= 8230.0
https://au.finance.yahoo.com/quote/CDA.AX/cash-flow?p=CDA.AX
ca

...

cashburn= 3117.0
https://au.finance.yahoo.com/quote/NAN.AX/balance-sheet?p=NAN.AX
curcash= 48841.0
https://au.finance.yahoo.com/quote/NAC.AX/cash-flow?p=NAC.AX
cashburn= -3924.88
https://au.finance.yahoo.com/quote/NAC.AX/balance-sheet?p=NAC.AX
curcash= 2169.25
code NAC cashburn too much and dropped
https://au.finance.yahoo.com/quote/NSC.AX/cash-flow?p=NSC.AX
cashburn= 180.0
https://au.finance.yahoo.com/quote/NSC.AX/balance-sheet?p=NSC.AX
curcash= 9436.0
https://au.finance.yahoo.com/quote/NSR.AX/cash-flow?p=NSR.AX
cashburn= 9792.0
https://au.finance.yahoo.com/quote/NSR.AX/balance-sheet?p=NSR.AX
curcash= 23166.0
https://au.finance.yahoo.com/quote/NVL.AX/cash-flow?p=NVL.AX
cashburn= 4109.0
https://au.finance.yahoo.com/quote/NVL.AX/balance-sheet?p=NVL.AX
curcash= 4301.0
https://au.finance.yahoo.com/quote/NEA.AX/cash-flow?p=NEA.AX
cashburn= -4980.0
https://au.finance.yahoo.com/quote/NEA.AX/balance-sheet?p=NEA.AX
curcash= 12189.0
https://au.finance.yahoo.com/quote/NTC.AX/cash-flow?p=NTC.AX
c

curcash= 9939.94
https://au.finance.yahoo.com/quote/PGY.AX/cash-flow?p=PGY.AX
cashburn= -699.57
https://au.finance.yahoo.com/quote/PGY.AX/balance-sheet?p=PGY.AX
curcash= 635.62
code PGY cashburn too much and dropped
https://au.finance.yahoo.com/quote/PNI.AX/cash-flow?p=PNI.AX
cashburn= -2599.0
https://au.finance.yahoo.com/quote/PNI.AX/balance-sheet?p=PNI.AX
curcash= 10945.0
https://au.finance.yahoo.com/quote/PNC.AX/cash-flow?p=PNC.AX
cashburn= 271.0
https://au.finance.yahoo.com/quote/PNC.AX/balance-sheet?p=PNC.AX
curcash= 3410.0
https://au.finance.yahoo.com/quote/PGM.AX/cash-flow?p=PGM.AX
cashburn= 2435.65
https://au.finance.yahoo.com/quote/PGM.AX/balance-sheet?p=PGM.AX
curcash= 3331.59
https://au.finance.yahoo.com/quote/PL8.AX/cash-flow?p=PL8.AX
cashburn= 750.0
https://au.finance.yahoo.com/quote/PL8.AX/balance-sheet?p=PL8.AX
curcash= 900.0
https://au.finance.yahoo.com/quote/PLC.AX/cash-flow?p=PLC.AX
cashburn= 170.33
https://au.finance.yahoo.com/quote/PLC.AX/balance-sheet?p=PLC.AX
curc

curcash= 17406.0
https://au.finance.yahoo.com/quote/REF.AX/cash-flow?p=REF.AX
cashburn= -1972.38
https://au.finance.yahoo.com/quote/REF.AX/balance-sheet?p=REF.AX
curcash= 5313.4
https://au.finance.yahoo.com/quote/RWD.AX/cash-flow?p=RWD.AX
cashburn= 2287.43
https://au.finance.yahoo.com/quote/RWD.AX/balance-sheet?p=RWD.AX
curcash= 6299.35
https://au.finance.yahoo.com/quote/RXH.AX/cash-flow?p=RXH.AX
cashburn= -3952.47
https://au.finance.yahoo.com/quote/RXH.AX/balance-sheet?p=RXH.AX
curcash= 906.53
code RXH cashburn too much and dropped
https://au.finance.yahoo.com/quote/RNO.AX/cash-flow?p=RNO.AX
cashburn= 1244.14
https://au.finance.yahoo.com/quote/RNO.AX/balance-sheet?p=RNO.AX
curcash= 2612.76
https://au.finance.yahoo.com/quote/RMY.AX/cash-flow?p=RMY.AX
cashburn= 422.0
https://au.finance.yahoo.com/quote/RMY.AX/balance-sheet?p=RMY.AX
curcash= 1098.0
https://au.finance.yahoo.com/quote/RMG.AX/cash-flow?p=RMG.AX
cashburn= 1167.92
https://au.finance.yahoo.com/quote/RMG.AX/balance-sheet?p=RMG.A

cashburn= -52532.0
https://au.finance.yahoo.com/quote/TFL.AX/balance-sheet?p=TFL.AX
curcash= 2799.0
code TFL cashburn too much and dropped
https://au.finance.yahoo.com/quote/TGR.AX/cash-flow?p=TGR.AX
cashburn= -803.0
https://au.finance.yahoo.com/quote/TGR.AX/balance-sheet?p=TGR.AX
curcash= 12521.0
https://au.finance.yahoo.com/quote/TAW.AX/cash-flow?p=TAW.AX
cashburn= 9416.0
https://au.finance.yahoo.com/quote/TAW.AX/balance-sheet?p=TAW.AX
curcash= 16375.0
https://au.finance.yahoo.com/quote/TDL.AX/cash-flow?p=TDL.AX
cashburn= -2723.79
https://au.finance.yahoo.com/quote/TDL.AX/balance-sheet?p=TDL.AX
curcash= 7918.21
https://au.finance.yahoo.com/quote/TMP.AX/cash-flow?p=TMP.AX
cashburn= -632.81
https://au.finance.yahoo.com/quote/TMP.AX/balance-sheet?p=TMP.AX
curcash= 5601.35
https://au.finance.yahoo.com/quote/TCN.AX/cash-flow?p=TCN.AX
cashburn= -1837.01
https://au.finance.yahoo.com/quote/TCN.AX/balance-sheet?p=TCN.AX
curcash= 3115.67
https://au.finance.yahoo.com/quote/TPW.AX/cash-flow?p=TP

cashburn= -3914.0
https://au.finance.yahoo.com/quote/WLL.AX/balance-sheet?p=WLL.AX
curcash= 7764.0
https://au.finance.yahoo.com/quote/WWI.AX/cash-flow?p=WWI.AX
cashburn= -187.0
https://au.finance.yahoo.com/quote/WWI.AX/balance-sheet?p=WWI.AX
curcash= 412.0
https://au.finance.yahoo.com/quote/WHC.AX/cash-flow?p=WHC.AX
cashburn= -940.0
https://au.finance.yahoo.com/quote/WHC.AX/balance-sheet?p=WHC.AX
curcash= 101453.0
https://au.finance.yahoo.com/quote/WND.AX/cash-flow?p=WND.AX
cashburn= -408.25
https://au.finance.yahoo.com/quote/WND.AX/balance-sheet?p=WND.AX
curcash= 704.45
https://au.finance.yahoo.com/quote/WNR.AX/cash-flow?p=WNR.AX
cashburn= 1431.64
https://au.finance.yahoo.com/quote/WNR.AX/balance-sheet?p=WNR.AX
curcash= 2144.93
https://au.finance.yahoo.com/quote/WFE.AX/cash-flow?p=WFE.AX
no information in cash flow report.
https://au.finance.yahoo.com/quote/WTC.AX/cash-flow?p=WTC.AX
cashburn= 66372.0
https://au.finance.yahoo.com/quote/WTC.AX/balance-sheet?p=WTC.AX
curcash= 109527.0
ht